### Multiple Producers and Consumers with Semaphores (C) [4 points]

#### Producers and Consumers

Here is a producer/consumer problem with semaphores in C. The producer produces the first `numIters` numbers; the consumer expects `numIters` numbers and prints their sum. The shared variable `data` is declared as `volatile`. This tells the compiler that the variable can change at any time and has to be read from memory. Without that, in the function `Consumer`, the compiler may assume that `data` does not change in the function and keep it in a register.

In [1]:
%%writefile pc.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <semaphore.h>
#define SHARED 1

sem_t empty, full;                /* the global semaphores */
volatile int data;                /* shared buffer         */
int numIters;

/* deposit 1, ..., numIters into the data buffer */
void *Producer(void *arg) {
    printf("Producer created\n");
    for (int produced = 0; produced < numIters; produced++) {
        sem_wait(&empty);
        data = produced;
        sem_post(&full);
    }
}

/* fetch numIters items from the buffer and sum them */
void *Consumer(void *arg) {
    printf("Consumer created\n");
    int sum = 0;
    for (int consumed = 0; consumed < numIters; consumed++) {
        sem_wait(&full);
        sum += data;
        sem_post(&empty);
    }
    printf("For %d iterations, the sum is %d\n", numIters, sum);
}

/* main program: read command line and create threads */
int main(int argc, char *argv[]) {
    pthread_t pid, cid;           /* thread and attributes */
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);
    sem_init(&empty, SHARED, 1);  /* sem empty = 1 */
    sem_init(&full, SHARED, 0);   /* sem full = 0  */
    numIters = atoi(argv[1]);
    pthread_create(&pid, &attr, Producer, NULL);
    pthread_create(&cid, &attr, Consumer, NULL);
    pthread_join(pid, NULL);
    pthread_join(cid, NULL);
}

Overwriting pc.c


Compile and run the code. The numbers from 0 to the command line parameter are produced and consumed. The consumer prints the sum of the consumed numbers.

In [2]:
!gcc pc.c -lpthread -o pc

In [3]:
!./pc 6

Producer created
Consumer created
For 6 iterations, the sum is 15


#### Multiple Producers and Consumers

Generalize the above C implementation for an arbitrary number of producers/consumers. The program should take two parameters from the command line: the number of iterations of each producer/consumer and the number of producers/consumers. All producers and all consumers have the same number of iterations, and there are as many consumers as there are producers. The output should be modified such that each producer and consumer identifies itself when printing that they were created and when printing their sum. The main program should additionally print the expected total sum by computing that, not by collecting the sums of each consumer: the sum each consumer computes may be different now due to nondeterminism, but their total sum is unique. Recall that the sum of the first `n - 1` numbers is `n × (n - 1) / 2`.  For example
```bash
!./pc 10 3
```
may result in:
```
Producer 1 created
Producer 0 created
Consumer 0 created
Producer 2 created
Consumer 1 created
Consumer 2 created
For 10 iterations, the sum of consumer 0 is 36
For 10 iterations, the sum of consumer 1 is 45
For 10 iterations, the sum of consumer 2 is 54
The expected total sum is 135
```
*Hint:* Copy and modify the above code. When creating producer and consumer threads, the number of the producer or consumer can be passed to the producer or consumer as the last parameter of `pthread_create`, using appropriate type casts. Inside each producer and consumer, the `arg` parameter has to be cast back to an integer.

In [4]:
%%writefile pc.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <semaphore.h>
#define SHARED 1

sem_t empty, full;                /* the global semaphores */
volatile int data;                /* shared buffer         */
int numIters;

/* deposit 1, ..., numIters into the data buffer */
void *Producer(void *arg) {
    int producerID = *((int*)arg);
    printf("Producer %d created\n", producerID);
    for (int produced = 0; produced < numIters; produced++) {
        sem_wait(&empty);
        data = produced;
        sem_post(&full);
    }
}

/* fetch numIters items from the buffer and sum them */
void *Consumer(void *arg) {
    int consumerID = *((int*)arg);
    printf("Consumer %d created\n", consumerID);
    int sum = 0;
    for (int consumed = 0; consumed < numIters; consumed++) {
        sem_wait(&full);
        sum += data;
        sem_post(&empty);
    }
    printf("For %d iterations, the sum of consumer %d is %d\n", numIters, consumerID, sum);
}

/* main program: read command line and create threads */
int main(int argc, char *argv[]) {
    int n = atoi(argv[2]);
    pthread_t pid[n], cid[n];           /* thread and attributes */
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);
    sem_init(&empty, SHARED, 1);  /* sem empty = 1 */
    sem_init(&full, SHARED, 0);   /* sem full = 0  */
    numIters = atoi(argv[1]);

    int producerIDs[n];
    int consumerIDs[n];
    for (int i = 0; i < n; i ++) {
    producerIDs[i] = i;
    consumerIDs[i] = i;
    pthread_create(&pid[i], &attr, Producer, (void*)&producerIDs[i]);
    pthread_create(&cid[i], &attr, Consumer, (void*)&consumerIDs[i]);
    }
    for (int i = 0; i < n; i++) {
    pthread_join(pid[i], NULL);
    pthread_join(cid[i], NULL);
    }

    int expectedSum = (numIters * (numIters - 1) / 2) * n;
    printf("The expected total sum is %d\n", expectedSum);
}

Overwriting pc.c


In [5]:
!gcc pc.c -lpthread -o pc

In [6]:
!./pc 10 5

Consumer 3 created
Consumer 1 created
Consumer 4 created
Producer 0 created
Producer 3 created
Producer 2 created
Producer 1 created
Consumer 0 created
Consumer 2 created
For 10 iterations, the sum of consumer 3 is 41
For 10 iterations, the sum of consumer 0 is 49
Producer 4 created
For 10 iterations, the sum of consumer 1 is 39
For 10 iterations, the sum of consumer 4 is 45
For 10 iterations, the sum of consumer 2 is 51
The expected total sum is 225


**Bonus:** The main program should collect the sums computed by each consumer and print a line with

    The computed total sum is ...
    
For this, have each consumer `return` its sum. Avoid the use of global variables. Your program must compile without warnings. [4 bonus points]

In [7]:
%%writefile pc.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <semaphore.h>
#define SHARED 1

sem_t empty, full;                /* the global semaphores */
volatile int data;                /* shared buffer         */
int numIters;

/* deposit 1, ..., numIters into the data buffer */
void *Producer(void *arg) {
    int producerID = *((int*)arg);
    printf("Producer %d created\n", producerID);
    for (int produced = 0; produced < numIters; produced++) {
        sem_wait(&empty);
        data = produced;
        sem_post(&full);
    }
}

/* fetch numIters items from the buffer and sum them */
void *Consumer(void *arg) {
    int consumerID = *((int*)arg);
    printf("Consumer %d created\n", consumerID);
    int sum = 0;
    for (int consumed = 0; consumed < numIters; consumed++) {
        sem_wait(&full);
        sum += data;
        sem_post(&empty);
    }
    printf("For %d iterations, the sum of consumer %d is %d\n", numIters, consumerID, sum);

    int* result = malloc(sizeof(int));
    *result = sum;
    return (void *) result;
}

/* main program: read command line and create threads */
int main(int argc, char *argv[]) {
    int n = atoi(argv[2]);
    pthread_t pid[n], cid[n];           /* thread and attributes */
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);
    sem_init(&empty, SHARED, 1);  /* sem empty = 1 */
    sem_init(&full, SHARED, 0);   /* sem full = 0  */
    numIters = atoi(argv[1]);

    int producerIDs[n];
    int consumerIDs[n];
    int total = 0;

    for (int i = 0; i < n; i ++) {
    producerIDs[i] = i;
    consumerIDs[i] = i;
    pthread_create(&pid[i], &attr, Producer, (void*)&producerIDs[i]);
    pthread_create(&cid[i], &attr, Consumer, (void*)&consumerIDs[i]);
    }

    int* val;
    for (int i = 0; i < n; i++) {
    pthread_join(pid[i], NULL);
    pthread_join(cid[i], (void**) &val);
    total = total + *val;
    free(val);
    }
    
    int expectedSum = (numIters * (numIters - 1) / 2) * n;
    printf("The expected total sum is %d\n", expectedSum);
    printf("The computed total sum is %d\n", total);
}

/* video used for help: https://www.youtube.com/watch?v=ln3el6PR__Q */

Overwriting pc.c


In [8]:
!gcc pc.c -lpthread -o pc

In [9]:
!./pc 10 5

Producer 1 created
Producer 0 created
Producer 2 created
Consumer 1 created
For 10 iterations, the sum of consumer 1 is 12
Consumer 2 created
Consumer 0 created
Producer 3 created
For 10 iterations, the sum of consumer 2 is 55
For 10 iterations, the sum of consumer 0 is 47
Consumer 4 created
For 10 iterations, the sum of consumer 4 is 66
Producer 4 created
Consumer 3 created
For 10 iterations, the sum of consumer 3 is 45
The expected total sum is 225
The computed total sum is 225
